In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
# ------------------------
# 1. Load the Data
# ------------------------
df = pd.read_excel("Brent_Calendar_Price.xlsx", header=1)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.sort_values("Date")
df.set_index("Date", inplace=True)

# ------------------------
# Filter for post-2020
# ------------------------
df = df[df.index.year > 2024]

# ------------------------
# 2. Rename Columns to CO1, CO2, ..., CO13
# ------------------------
rename_map = {col: f"CO{i+1}" for i, col in enumerate(df.columns)}
df.rename(columns=rename_map, inplace=True)
n_contracts = sum(col.startswith("CO") for col in df.columns)

# ------------------------
# 3. Compute All Calendar Spreads (COi - COj for i < j)
# ------------------------
spread_cols = []
for i in range(1, n_contracts+1):
    for j in range(i+1, n_contracts+1):
        col_name = f"Spread_{i}_{j}"
        df[col_name] = df[f"CO{i}"] - df[f"CO{j}"]
        spread_cols.append(col_name)

# ------------------------
# 4. Interactive Plot of Key Spreads
# ------------------------
key_spreads = ["Spread_1_3", "Spread_3_6", "Spread_6_9", "Spread_9_12"]
fig = go.Figure()

for spread in key_spreads:
    if spread in df.columns:
        fig.add_trace(go.Scatter(
            x=df.index, 
            y=df[spread], 
            mode='lines',
            name=spread
        ))

fig.add_hline(y=0, line_dash="dash", line_color="black")
fig.update_layout(
    title="Brent Calendar Spreads (Post-2020)",
    xaxis_title="Date",
    yaxis_title="Spread ($/bbl)",
    template="plotly_white",
    hovermode="x unified"
)
fig.show()

# ------------------------
# Multi-Date Term Structure (Curve Snapshot)
# ------------------------
def plot_term_structures(dates):
    months = list(range(1, n_contracts+1))
    fig = px.line(
        labels={'x': 'Contract Month', 'y': 'Price ($/bbl)', 'color': 'Date'},
        title="Brent Futures Term Structures"
    )

    for date in dates:
        try:
            row = df.loc[date, [f"CO{i}" for i in range(1, n_contracts+1)]]
        except KeyError:
            print(f"Date {date} not found in dataset.")
            continue
        
        fig.add_scatter(
            x=months,
            y=row.values,
            mode="lines+markers",
            name=str(date.date())
        )

    fig.update_layout(template="plotly_white")
    fig.show()


# Example usage with multiple dates
plot_term_structures([
    pd.Timestamp('2025-04-29'),
    pd.Timestamp('2025-05-01'),
    pd.Timestamp('2025-05-01')
])








In [34]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ------------------------
# 4b. Interactive Plot of Key Spreads + CO1 on dual y-axis
# ------------------------
fig_dual = make_subplots(specs=[[{"secondary_y": True}]])

# Add spread traces to primary y-axis
for spread in key_spreads:
    if spread in df.columns:
        fig_dual.add_trace(
            go.Scatter(
                x=df.index,
                y=df[spread],
                mode='lines',
                name=spread
            ),
            secondary_y=False
        )

# Add CO1 to secondary y-axis
fig_dual.add_trace(
    go.Scatter(
        x=df.index,
        y=df["CO1"],
        mode='lines',
        name="CO1",
        line=dict(color='red', width=2)
    ),
    secondary_y=True
)

# Add zero line for spreads
fig_dual.add_hline(y=0, line_dash="dash", line_color="black")

# Update layout
fig_dual.update_layout(
    title="Brent Calendar Spreads (Post-2020) with CO1",
    xaxis_title="Date",
    template="plotly_white",
    hovermode="x unified"
)

# Set y-axis titles
fig_dual.update_yaxes(title_text="Spread ($/bbl)", secondary_y=False)
fig_dual.update_yaxes(title_text="CO1 Price ($/bbl)", secondary_y=True)

fig_dual.show()


In [39]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

# ============================
# Spread Analysis Function
# ============================
def spread_analysis(df, spread, window=30):
    if spread not in df.columns:
        print(f"Spread {spread} not found in dataframe.")
        return

    # --- Rolling Mean & Volatility ---
    df[f"{spread}_roll_mean"] = df[spread].rolling(window).mean()
    df[f"{spread}_roll_std"] = df[spread].rolling(window).std()

    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=df.index, y=df[spread], mode="lines", name=spread))
    fig1.add_trace(go.Scatter(x=df.index, y=df[f"{spread}_roll_mean"], mode="lines", name=f"{window}D Mean"))
    fig1.add_trace(go.Scatter(x=df.index, y=df[f"{spread}_roll_mean"] + 2*df[f"{spread}_roll_std"], 
                             mode="lines", name="+2σ", line=dict(dash="dot")))
    fig1.add_trace(go.Scatter(x=df.index, y=df[f"{spread}_roll_mean"] - 2*df[f"{spread}_roll_std"], 
                             mode="lines", name="-2σ", line=dict(dash="dot")))
    fig1.update_layout(title=f"{spread} Rolling Mean & Volatility", template="plotly_white")
    fig1.show()

    # --- Distribution ---
    fig2 = ff.create_distplot([df[spread].dropna()], [spread], show_hist=True, show_rug=False)
    fig2.update_layout(title=f"{spread} Distribution", template="plotly_white")
    fig2.show()

    # --- Percentiles ---
    percentiles = df[spread].quantile([0.05, 0.5, 0.95])
    print("\nPercentiles:")
    print(percentiles)

    fig3 = go.Figure()
    fig3.add_trace(go.Scatter(x=df.index, y=df[spread], mode="lines", name=spread))
    for q, val in percentiles.items():
        fig3.add_hline(y=val, line_dash="dot", annotation_text=f"{int(q*100)}%")
    fig3.update_layout(title=f"{spread} with Percentile Bands", template="plotly_white")
    fig3.show()

    # --- Regime Detection ---
    df["Regime"] = df[spread].apply(lambda x: "Backwardation" if x > 0 else "Contango")
    fig4 = go.Figure()
    fig4.add_trace(go.Scatter(x=df.index, y=df[spread], mode="lines", name=spread))
    fig4.add_hline(y=0, line_dash="dash", line_color="black")
    fig4.update_layout(title=f"{spread} Regime Analysis", template="plotly_white")
    fig4.show()

    print("\nRegime distribution:")
    print(df["Regime"].value_counts(normalize=True))


# ============================

# ============================
spread_analysis(df, "Spread_9_12")



Percentiles:
0.05   -0.380
0.50    0.330
0.95    1.149
Name: Spread_9_12, dtype: float64



Regime distribution:
Regime
Backwardation    0.660377
Contango         0.339623
Name: proportion, dtype: float64


In [41]:
import plotly.express as px
import plotly.graph_objects as go

# ============================
# Multi-Spread Comparison
# ============================
def multispread_comparison(df, spread_cols, pair_compare=None):
    # --- Correlation Matrix ---
    corr = df[spread_cols].corr()
    fig1 = px.imshow(
        corr,
        text_auto=".2f",
        color_continuous_scale="RdBu_r",
        title="Spread Correlation Matrix"
    )
    fig1.update_layout(template="plotly_white")
    fig1.show()

    # --- Heatmap of Spreads Over Time ---
    spread_matrix = df[spread_cols]
    fig2 = px.imshow(
        spread_matrix.T,
        aspect="auto",
        color_continuous_scale="RdBu_r",
        labels={"x": "Date", "y": "Spread", "color": "$/bbl"},
        title="Brent Calendar Spread Heatmap"
    )
    fig2.update_layout(template="plotly_white")
    fig2.show()

    # --- Optional Pair Scatter ---
    if pair_compare and all(sp in df.columns for sp in pair_compare):
        fig3 = px.scatter(
            df, x=pair_compare[0], y=pair_compare[1],
            title=f"{pair_compare[0]} vs {pair_compare[1]}",
            trendline="ols"
        )
        fig3.update_layout(template="plotly_white")
        fig3.show()


# ============================
# Example Usage
# ============================
key_spreads = ["Spread_1_3", "Spread_3_6", "Spread_6_9", "Spread_9_12"]
multispread_comparison(df, key_spreads, pair_compare=("Spread_1_3", "Spread_9_12"))


In [35]:
import pandas as pd
import plotly.express as px

# ------------------------
# Prepare list of spreads
# ------------------------
data = []
for i in range(1, 13):
    for j in range(i, 13):
        col = f'Spread_{i}_{j}'
        if col in df.columns:
            corr = df[col].corr(df['CO1'])
        else:
            corr = None
        data.append({'i': i, 'j': j, 'Correlation': corr})

# ------------------------
# Create DataFrame
# ------------------------
corr_df = pd.DataFrame(data)

# ------------------------
# Pivot for heatmap
# ------------------------
heatmap_df = corr_df.pivot(index='i', columns='j', values='Correlation')

# ------------------------
# Plot heatmap
# ------------------------
fig = px.imshow(
    heatmap_df,
    text_auto=True,
    color_continuous_scale='RdBu_r',
    zmin=-1,
    zmax=1,
    labels=dict(x="j", y="i", color="Correlation"),
    title="Correlation between CO1 and Spreads"
)

fig.show()
